# 🏭 Industria y Manufactura: OEE, calidad y throughput
KPIs: OEE, scrap, MTBF/MTTR, lead time, first pass yield.

## Casos de uso
- Control estadístico de procesos (SPC)
- Mantenimiento predictivo y calidad visual
- Gemelos digitales y simulación de cuellos de botella
- Trazabilidad de lote y recall

In [ ]:
# Cálculo simple de OEE
def oee(disponibilidad, rendimiento, calidad):
    return disponibilidad * rendimiento * calidad
round(oee(0.9, 0.92, 0.98), 3)

## Arquitectura
- Ingesta PLC/SCADA/IoT, series de tiempo
- Modelos de visión y features en borde (edge)
- Integración MES/ERP y tableros de planta